# Week 1: Adding unit model to flowsheet

This tutorial will go through the basic workflow to create a WaterTAP model with a single unit proces (pump).

1. Create model
2. Add flowsheet
3. Add property model
4. Add pump model
5. Set operating conditions
6. Scaling model
7. Initialize model
8. Solve model

## Step 0: Import necessary packages

### Any WaterTAP model will begin with importing the relevant components from Pyomo, IDAES, and WaterTAP.

From Pyomo:
- Concrete model object
- units 
- Helper function(s) to check solver status

From IDAES:
- Flowsheet block
- Helper function to check degrees of freedom
- Helper function to scale model

From WaterTAP:
- Property models
- Unit models
- Solver


> Note: Both `ConcreteModel` and `FlowsheetBlock` are required imports for any WaterTAP model.


In [ ]:
# Pyomo imports
from pyomo.environ import (
    ConcreteModel,
    check_optimal_termination,
    assert_optimal_termination,
    value,
    units as pyunits,
)

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump


## Step 1: Create base model

Pyomo's `ConcreteModel` is the basis for any WaterTAP model.

No arguments are required. Convention is to call this `m` but any local name can be used.

In [ ]:
m = ConcreteModel()

## Step 2: Add flowsheet block

The [`FlowsheetBlock`](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/flowsheet_block.html#) from IDAES forms the base for all WaterTAP system models.

All model components (unit models, property models, variables, constraints, etc.) are built upon the flowsheet block.

The flowsheet block will create the time domain for the flowsheet. WaterTAP is a steady-state modeling framework, so we set `dynamic=False` when creating the `FlowsheetBlock`.

(Note: `dynamic=False` is the default argument, but excluding it will raise a warning.)

In [ ]:
m.fs = FlowsheetBlock(dynamic=False)

# Step 3: Add property model

With the flowsheet created, we can add any sub-models required for the WaterTAP system model.

The property model used will be dictated by the models on the flowsheet. In this demonstration, we are using the WaterTAP [seawater property model](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/seawater.html).

Creating the `SeawaterParameterBlock` does not require any arguments. However, other property models can have many required and optional arguments to specify the model.

In [ ]:
m.fs.properties = SeawaterParameterBlock()

# Step 4: Add unit model

After adding the property model, we can add the unit models to the flowsheet. In this demonstration, we are adding a single [pump model](https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/pump.html).

Adding any unit model requires specifying the property package via the `property_package` argument. Here, we are specifying that the property model to be used with our pump model is `m.fs.properties` that we just added to the flowsheet.

In [ ]:
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Step 5: Set operating conditions

Now that we have added all the necessary models to the flowsheet, we can specify the operating conditions.

First, we check the degrees of freedom that need to be specified prior to solving the model.


In [ ]:
print("Degrees of freedom =", degrees_of_freedom(m))

### The degrees of freedom for the pump model include:
- the change in pressure from inlet to outlet
- pump efficiency
- inlet stream state variables

The inlet stream conditions to any unit model are defined by the property model state variables. For the pump model, the inlet and outlet properties are managed by a [control volume](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/control_volume_0d.html#module-idaes.core.base.control_volume0d).

We can use the `.display()` method on the pump unit model control volume. Using this method on any block will automatically go through all the sub-blocks and show all the variables, constraints, and objectives currently on each block.

In [ ]:
m.fs.pump.control_volume.display()

### This reveals that upon `m.fs.pump.control_volume` are two sub-blocks called `properties_in` and `properties_out`. 

Each of these are indexed to `0`, which is the default time-index for steady-state models.

To define the inlet stream to our pump, we only need to specify the state variables for `properties_in`. The state variables are defined by the property model used for the unit model (`SeawaterParameterBlock` here) but are nearly always:
- `pressure`: pressure of inlet stream (Pa)
- `temperature`: temperature of (K)
- `flow_mass_phase_comp["Liq", "H2O"]`: mass flow rate of water (kg/s)
- `flow_mass_phase_comp["Liq", "TDS"]`: mass flow rate of TDS (kg/s)

> Note: The naming of property variables in WaterTAP property models follow the [naming conventions used in IDAES](https://idaes-pse.readthedocs.io/en/latest/explanations/conventions.html#standard-naming-format).

In [ ]:
m.fs.pump.control_volume.properties_in[0].pressure.fix(101325)  # Pa
m.fs.pump.control_volume.properties_in[0].temperature.fix(298.15)  # K
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(1) # kg/s
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.1) # kg/s

print("Degrees of freedom =", degrees_of_freedom(m))

### With the inlet stream specified, the remaining two degrees of freedom are variables that are built directly on the pump block itself (rather than on the control volume sub-block):

In [ ]:
m.fs.pump.deltaP.fix(500000)  # Pa
m.fs.pump.efficiency_pump.fix(0.8)

print("Degrees of freedom =", degrees_of_freedom(m))

# Step 6: Scaling the model

Prior to model initialization and solving, the variables and constraints on the model must be properly scaled.

Model scaling is a topic that will be covered in more detail in a future class, but here we present the helper function `calculate_scaling_factors` that will use default scaling factors to scale the variables on the model.

> Note: Since we did not set default scaling factors for some of our variables, this will result in printed warnings.



In [ ]:
calculate_scaling_factors(m)

# Step 7: Initializing the model

Before solving the model, we run an initialization routine that will set model variables to initial values based on the operating conditions we have specified in the previous step.

Every model has a different initialization routine that was implemented by the model developer, but they can all be called with the `initialize()` method.

In [ ]:
m.fs.pump.initialize()

# Step 8: Solving the model

After initialization, we are ready to solve the model.

Solving is done by passing the model to a *solver*. WaterTAP uses a custom implementation of the IPOPT solver. An instance of the WaterTAP solver is returned from the `get_solver()` function that was imported above.

In [ ]:
solver = get_solver()
results = solver.solve(m)

### Checking the termination status of the solve

We want to know if the solve resulted in an optimal termination. This can be done in a few ways:
- using the `assert_optimal_termination` or `check_optimal_termination` function
- printing the solver status from the `results`

In [ ]:
# This will raise an error if the model did not solve optimally
assert_optimal_termination(results)

# Alternatively, we can check the termination status without raising an error
termination_status = check_optimal_termination(results)
print("Optimal termination:", termination_status)

# Printing termination condition from results
print("Solver Termination Condition:", results.solver.termination_condition)

# Accessing results

The values of all variables on the model can be accessed directly by:
- using the `value()` function
- accessing the `.value` attribute
- by directly calling the variable

In [ ]:
pump_work1 = value(m.fs.pump.work_mechanical[0])
pump_work2 = m.fs.pump.work_mechanical[0].value
pump_work3 = m.fs.pump.work_mechanical[0]()

print(f"Pump work (method 1): {pump_work1:.2f} W")
print(f"Pump work (method 2): {pump_work2:.2f} W")
print(f"Pump work (method 3): {pump_work3:.2f} W")

# Other stuff?

With the basics for building a unit model flowsheet in WaterTAP, the rest of this tutorial will highlight some other stuff.

First, we put all the steps for creating our model into a single `build` function that returns the model.

In [ ]:
def build():
    m = ConcreteModel()

    m.fs = FlowsheetBlock(dynamic=False)

    m.fs.properties = SeawaterParameterBlock()

    m.fs.pump = Pump(
        property_package=m.fs.properties,
    )

    calculate_scaling_factors(m)

    return m

# Using `calculate_state` method

Rather than directly fixing the state variables on the inlet state block `m.fs.pump.control_volume.properties_in`, users can define the state variables using other non-state variables.

For example, in water treatment, users might be more familiar with an inlet volumetric flow rate of water (`flow_vol_phase["Liq"]`) and TDS concentration (`conc_mass_phase_comp["Liq", "TDS"]`) rather than the mass flow of water and TDS.

The `calculate_state` method will use these two variables to calculate the inlet mass flows for water and TDS.

In [ ]:
m = build()

print(f"Degrees of freedom before calculate_state: {degrees_of_freedom(m)}")
m.fs.pump.control_volume.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", "Liq"): 1, # m³/s
        ("conc_mass_phase_comp", ("Liq", "TDS")): 0.5, # kg/m³
        ("pressure", None): 101325, # Pa
        ("temperature", None): 298, # K
    },
    hold_state=True,
)
print(f"Degrees of freedom after calculate_state: {degrees_of_freedom(m)}")

m.fs.pump.deltaP.fix(500000)  # Pa
m.fs.pump.efficiency_pump.fix(0.8)

print("Degrees of freedom =", degrees_of_freedom(m))

m.fs.pump.initialize()
solver = get_solver()
results = solver.solve(m)
assert_optimal_termination(results)

# Using `pyunits`

WaterTAP makes use of a package called [Pint](https://pint.readthedocs.io/en/stable/) to handle dimensional quantities, which we have imported as `pyunits`.

Variables, parameters, and expressions in WaterTAP models have physical units associated with them to ensure dimensional consistency.

But `pyunits` can be used just as easily to define a quantity outside of WaterTAP. A quantity is the product of the magnitude and the associated dimensional units.

In [71]:
conc = 35 * pyunits.g / pyunits.L
flow_vol = 1e6 * pyunits.gallons / pyunits.day

print(flow_vol)
# You can also use common prefixes to the units to indicate order of magnitude
# e.g., u = "micro" = 1E-6
flow_vol = 1 * pyunits.ugallons / pyunits.day
print(flow_vol)
# M = "mega" = 1E6
flow_vol = 1 * pyunits.Mgallons / pyunits.day
print(flow_vol)

1000000.0*gal/d
µgal/d
Mgal/d


### Converting units

You can convert from one set of units to another using the `convert` method and access the units of a quantity with the `get_units` method

In [83]:
flow_vol2 = pyunits.convert(flow_vol, to_units=pyunits.m**3 / pyunits.s)
print(f"Flow volume in MGD = {value(flow_vol):.2f} {pyunits.get_units(flow_vol)}")
print(f"Flow volume in m3/s = {value(flow_vol2):.6f} {pyunits.get_units(flow_vol2)}")


Flow volume in MGD = 1.00 Mgal/d
Flow volume in m3/s = 0.043813 m**3/s


### Creating new quantities



In [ ]:

flow = 1 * pyunits.Mgallons / pyunits.day
flow = 1 * pyunits.ugla

In [ ]:
m.fs.pump.control_volume.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", "Liq"): 1,
        ("conc_mass_phase_comp", ("Liq", "TDS")): 0.5,
        ("pressure", None): 101325,
        ("temperature", None): 298,
    },
    hold_state=True,
)

# m.fs.feed.properties.calculate_state(
#         var_args={
#             ("flow_vol_phase", "Liq"): flow_in,  # m3/s
#             ("conc_mass_phase_comp", ("Liq", target_ion)): conc_mass_in,  # kg/m3
#             ("pressure", None): 101325,
#             ("temperature", None): 298,
#         },
#         hold_state=True,
#     )

In [ ]:
# Provide unit model inputs. For the pump we need to provide inlet flow, inlet pressure, outlet pressure, temperature and efficiency

# Add pyunits

feed_flow_mass = 1 # kg/s
feed_mass_frac_TDS = 0.035
feed_mass_frac_H2O = 1 - feed_mass_frac_TDS

m.fs.pump.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(
    feed_flow_mass * feed_mass_frac_TDS
)
m.fs.pump.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_flow_mass * feed_mass_frac_H2O
)

feed_pressure_in = 1e5 # Pa
feed_pressure_out = 5e5 # Pa
feed_temperature = 273.15 + 25 # K

m.fs.pump.inlet.pressure[0].fix(feed_pressure_in)
m.fs.pump.inlet.temperature[0].fix(feed_temperature)
m.fs.pump.outlet.pressure[0].fix(feed_pressure_out)

m.fs.pump.efficiency_pump.fix(0.75)

In [ ]:
m.fs.pump.inlet.display()

In [ ]:
# Lets check the degrees of freedom of the flowsheet! This should ideally be zero now.
degrees_of_freedom(m)

In [ ]:
# Let's initialize the flowsheet
m.fs.pump.initialize()

In [ ]:
# Let's solve the flowsheet now!
solver = get_solver()
results = solver.solve(m, tee=False)
# Ensure that the solver status is ok and optimal termination
assert_optimal_termination(results)

In [ ]:
def build():
    return m

m.fs.pump.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", ["Liq"]): m.fs.pump.inlet.flow_vol_phase[0, "Liq"],
        ("conc_mass_phase_comp", ["Liq", "TDS"]): m.fs.pump.inlet.conc_mass_phase_comp[0, "Liq", "TDS"],
        ("pressure"): feed_pressure_in,
        ("temperature"): feed_temperature,
    }
)